# LIC plots Cabral 1993

Written by Monika Moscibrodzka, Nov 2nd 2020

Source: github/moscibrodzka/ipole-lic

C version of the code on github/moscibrodzka/ipole-lic

In [295]:
import os
import sys
import matplotlib
import numpy as np
import ehtim as eh
import argparse
import re
import time
import ehtim.observing.obs_simulate as simobs
import pandas as pd 
from matplotlib import pyplot as plt
import glob
import math
import warnings
from matplotlib.ticker import MaxNLocator
import h5py
import matplotlib.gridspec as gridspec
from random import random
from random import seed
from tqdm import tqdm 

In [ ]:
path='/home/mmosc/eht/eht_software/current_plate/pol_data_plots/outreach/M87_final_polarimetric_average_images/'
file_name='M87_lo_3601_polarimetric_average_image_256.fits'

im=eh.image.load_fits(path+file_name,polrep='stokes')

lic_plot(im,L=2.,dl_step=1.0,PDF="lic.png")




Loading...:   0%|          | 0/256 [00:00<?, ?it/s]

Loading fits image:  /home/mmosc/eht/eht_software/current_plate/pol_data_plots/outreach/M87_final_polarimetric_average_images/M87_lo_3601_polarimetric_average_image_256.fits


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in double_scalars



Loading...:   2%|▏         | 6/256 [00:00<00:04, 54.89it/s]


Loading...:   5%|▍         | 12/256 [00:00<00:04, 54.56it/s]


Loading...:   7%|▋         | 18/256 [00:00<00:04, 55.94it/s]


Loading...:  10%|▉         | 25/256 [00:00<00:03, 57.86it/s]


Loading...:  12%|█▏        | 31/256 [00:00<00:03, 57.41it/s]


Loading...:  14%|█▍        | 37/256 [00:00<00:03, 57.28it/s]


Loading...:  17%|█▋        | 43/256 [00:00<00:03, 57.98it/s]


Loading...:  19%|█▉        | 49/256 [00:00<00:03, 57.36it/s]


Loading...:  21%|██▏       | 55/256 [00:00<00:03, 57.84it/s]


Loading...:  24%|██▍       | 62/256 [00:01<00:03, 58.99it/s]


Loading...:  27%|██▋       | 68/256 [00:01<00:03, 59.27it/s]


Loading...:  29%|██▉       | 74/256 [00:01<00:03, 59.27it/s]


Loading...:  32%|███▏      | 81/256 [00:01<00:02, 59.89it/s]


Loading...:  34%|███▍      | 87/256 [00:01<00:03, 55.49it/s]

In [303]:
# Author Monika Moscibrodzka Nov 2 2020
def lic_plot(im,L=20.,dl_step=0.5,PDF=None):

    self=im.copy()
    
    
    Imax = max(self.imvec)
    
    imarr = self.imvec.reshape(self.ydim, self.xdim)
    
    imageI = imarr.copy()

    
    seed(176545)
    for i in range(self.xdim):
        for j in range(self.ydim):
            imageI[i][j] *=random()
    
    ang=np.angle(self.qvec+1j*self.uvec)/2.       
    vx1 = -np.sin(ang+np.pi/2.);
    vy1 =  np.cos(ang+np.pi/2.);
    vx=vx1.reshape(self.ydim, self.xdim)
    vy=vy1.reshape(self.ydim, self.xdim)

    # generate 2 2d grids for the x & y bounds
    pixel=self.psize/eh.RADPERUAS #uas
    FOV=pixel*self.xdim
    x, y = np.mgrid[slice(-FOV/2, FOV/2, pixel),
                    slice(-FOV/2, FOV/2, pixel)]
    
    dl=dl_step                                                                                                                
    dlxy=np.sqrt(2.)*dl
    
    # boundries for integration
    startx=-FOV/2.;
    stopx=FOV/2.;
    starty=-FOV/2.;
    stopy=FOV/2.;

     
    # copy array structure
    image_lic=imarr.copy()
    image_lic_max=0.0
  
    
    # for each pixel perform LIC
    for i in tqdm (range (self.xdim), desc="Loading..."): 
        for j in range(self.ydim):
            image_lic[i][j]=0
            x_line=x[i][j]
            y_line=y[i][j]
            vx_line=vx[i][j]
            vy_line=vy[i][j]
            h=1./L                                                                                                                            
            totl1=0.0
            hsum=0.0
            #integrate forward until we hit boundary of image or integration limit                                                                                  
            while(totl1 < 0.5*L):
                x_line += vx_line*dl
                y_line += vy_line*dl
                if(x_line < startx or y_line < starty or x_line > stopx or y_line > stopy): break
                totl1 += dlxy
                hsum += h
                Ilocal=interp_scalar2d(x_line,y_line,imageI,startx,starty,pixel,pixel,self.xdim,self.ydim)
                image_lic[i][j] += Ilocal*h
                vx_line=interp_scalar2d(x_line,y_line,vx,startx,starty,pixel,pixel,self.xdim,self.ydim)
                vy_line=interp_scalar2d(x_line,y_line,vy,startx,starty,pixel,pixel,self.xdim,self.ydim)
            
            totl2=0.0
            x_line=x[i][j]
            y_line=y[i][j]
            vx_line=vx[i][j]
            vy_line=vy[i][j]
            while(totl2 < 0.5*L):
                x_line += -vx_line*dl
                y_line += -vy_line*dl
                if(x_line < startx or y_line < starty or x_line > stopx or y_line > stopy): break
                totl2 += dlxy
                hsum += h
                Ilocal=interp_scalar2d(x_line,y_line,imageI,startx,starty,pixel,pixel,self.xdim,self.ydim)
                image_lic[i][j] += Ilocal*h
                vx_line=interp_scalar2d(x_line,y_line,vx,startx,starty,pixel,pixel,self.xdim,self.ydim)
                vy_line=interp_scalar2d(x_line,y_line,vy,startx,starty,pixel,pixel,self.xdim,self.ydim)
            
            
            image_lic[i][j]/=hsum
            # find maximum to normalize
            if(image_lic[i][j] > image_lic_max): image_lic_max=image_lic[i][j]
            
    
    # merge lic plot with total intensity plot with wights ~ P^2 
    
    P2=self.qvec**2+self.uvec**2
    P2_max=max(P2)
    fac1=P2/P2_max
    fac=fac1.reshape(self.ydim, self.xdim)
    image_lic/=image_lic_max
    Ifinal=imarr/Imax*(1.-fac)+image_lic*fac
    

    # plot final result
    
    plt.figure(1,(7,7))
   
    plt.imshow(Ifinal,origin='upper',cmap='afmhot',vmin=0,vmax=1.2,
             extent=[np.min(x),np.max(x),np.min(y),np.max(y)])
    
    ax = plt.gca()
    empty_string_labels = ['']
    ax.set_xticklabels(empty_string_labels)
    ax.set_yticklabels(empty_string_labels)
    ax.tick_params(right= False,top= False,left= False, bottom= False)
    ax.set_aspect('equal')

    pic_fov=119
    ax.set_xlim(-pic_fov/2.,pic_fov/2.)
    ax.set_ylim(-pic_fov/2.,pic_fov/2.) 

    scale=True 
    if scale == True:
        
        scale_len=50.
        startx = -pic_fov/2. + 5 
        endx = -pic_fov/2. + scale_len +5
        
              
        starty=-48.
        endy=-48.
        plt.plot([startx, endx],
                 [starty, endy],
                 color="w", lw=3) # plot a line
        plt.text(x=(startx+endx)/2.0, y=starty-6., 
                 s= str(int(scale_len)) + " $\mu$as", color="w", 
                 ha="center", va="center",fontsize=40)
    
    
    # dump to file
    plt.tight_layout()
    if PDF !=None:
        plt.savefig(PDF)


    

In [298]:
#from ipole - interpolate vector or scalar to the point                                                                                                                          
def interp_scalar2d(xi,yi,var,startx,starty,dx,dy,NX,NY):


    (i,j,delx,dely) = xtoij(xi, yi,startx,starty,dx,dy,NX,NY)

    
    ip1 = i+1
    jp1 = j+1
    b1 = 1.-delx
    b2 = 1.-dely

    if(ip1 == NX):
        ip1=ip1-1
    if(jp1 == NY):
        jp1=jp1-1
     
    
    interp = var[i][j]*b1*b2 + var[i][jp1]*b1*dely + var[ip1][j]*delx*b2 + var[ip1][jp1]*delx*dely
    
    return(interp)




In [299]:
#from ipole - find index
def xtoij(xi, yi,startx,starty,dx,dy,NX,NY):

    
    #find index of pixel on the map                                                                                                                                           
    i = (int) ((xi - startx) / dx - 0.5 + 1000) - 1000;
    j = (int) ((yi - starty) / dy - 0.5 + 1000) - 1000;

    #and distance from boundaries                                                                                                                                   
    delx = (xi - ((i + 0.5) * dx + startx)) / dx
    if(i < 0):
        i = 0
        delx = 0. 
    if(i > NX-1):
        i = NX-1 
        delx = 1. 
    
    dely = (yi - ((j + 0.5) * dy + starty)) / dy
    if(j < 0):
        j = 0
        dely = 0. 
    if(j > NY-1):
        j = NY-1 
        dely = 1.     

    return (i,j,delx,dely)
